# Demo of Evaluation of Benchmarks

In [1]:
from dbmsbenchmarker import *
import pandas as pd

# Some plotly figures
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Some nice output
from IPython.display import display, Markdown

import logging
logging.basicConfig(level=logging.INFO)

## Inspect Result Folder

In [2]:
# path of folder containing experiment results
resultfolder = "/results/"

# create evaluation object for result folder
evaluate = inspector.inspector(resultfolder)

# list of all experiments in folder
evaluate.list_experiments
# dataframe of experiments
evaluate.get_experiments_preview()

,name,info,intro,queries,connections,time
1625144240,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,3,2021-07-01 15:17:46
1625159812,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,3,2021-07-01 19:37:26
1625170749,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,1,2021-07-01 20:35:16
1625255968,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,8,2021-07-02 23:51:22
1625242678,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,8,2021-07-02 19:38:33
1625238691,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,1,2021-07-02 16:05:19
1625180707,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,3,2021-07-02 01:26:50
1625203913,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H wit...,20,3,2021-07-02 05:58:21
1625172048,TPC-H Queries SF=10,This experiment compares run time and resource...,This includes the reading queries of TPC-H.,22,1,2021-07-01 20:56:29


## Pick an Experiment

In [3]:
# last Experiment
code = evaluate.list_experiments[len(evaluate.list_experiments)-1]

# Specific Experiment
code = '1625255968'

# load it
evaluate.load_experiment(code)

Benchmarking in folder /results//1625255968
Connections:
MariaDB-2-1
PostgreSQL-1-1
MonetDB-1-1
MySQL-1-1
MariaDB-1-1
MySQL-2-1
PostgreSQL-2-1
MonetDB-2-1
Queries:
0: Q1 = Pricing Summary Report (TPC-H Q1)
1: Q2 = Minimum Cost Supplier Query (TPC-H Q2)
2: Q3 = Shipping Priority (TPC-H Q3)
3: Q4 = Order Priority Checking Query (TPC-H Q4)
4: Q5 = Local Supplier Volume (TPC-H Q5)
5: Q6 = Forecasting Revenue Change (TPC-H Q6)
6: Q7 = Forecasting Revenue Change (TPC-H Q7)
7: Q8 = National Market Share (TPC-H Q8)
8: Q9 = Product Type Profit Measure (TPC-H Q9)
9: Q10 = Forecasting Revenue Change (TPC-H Q10)
10: Q11 = Important Stock Identification (TPC-H Q11)
11: Q12 = Shipping Modes and Order Priority (TPC-H Q12)
12: Q13 = Customer Distribution (TPC-H Q13)
13: Q14 = Forecasting Revenue Change (TPC-H Q14)
14: Q15 = Top Supplier Query (TPC-H Q15)
15: Q16 = Parts/Supplier Relationship (TPC-H Q16)
16: Q17 = Small-Quantity-Order Revenue (TPC-H Q17)
17: Q18 = Large Volume Customer (TPC-H Q18)
18: 

## Load general properties into variables

In [57]:
# get experiment workflow
df = evaluate.get_experiment_workflow()
#print(df)

# get workload properties
workload_properties = evaluate.get_experiment_workload_properties()
print(workload_properties['name'])
print(workload_properties['info'])

getWorkflow
TPC-H Queries SF=10
This experiment compares run time and resource consumption of TPC-H queries in different DBMS.


In [5]:
evaluator.pretty(evaluate.get_experiment_connection_properties('MonetDB-1-1'))

name:MonetDB-1-1
script:SF10-index
docker:MonetDB
docker_alias:Columnwise
version:11.37.11
info:[]
connectionmanagement
  numProcesses:1
  runsPerConnection:Unlimited
  timeout:3600
hostsystem
  RAM:1082007052288
  CPU:AMD EPYC 7742 64-Core Processor
  GPU:
  GPUIDs:[]
  Cores:256
  host:5.4.0-74-generic
  node:cl-worker28
  disk:623220008
  datadisk:10687696
  cuda:
  requests_cpu:4
  requests_memory:16Gi
  limits_cpu:0
  limits_memory:0
worker:[]
times
  load_ms:251632.57140340284
  benchmark_ms:316155.41979367845
prices
  perHour_usd:0.0
  benchmark_usd:0.0
hardwaremetrics
  total_cpu_memory:0
  total_cpu_memory_cached:0
  total_cpu_util:0
  total_gpu_util:0
  total_gpu_power:0
  total_gpu_memory:0
  total_cpu_throttled:0
  total_cpu_util_others:0
  total_cpu_util_s:0
  total_cpu_util_user_s:0
  total_cpu_util_sys_s:0
  total_cpu_throttled_s:0
  total_cpu_util_others_s:0
  total_network_rx:0
  total_network_tx:0
  total_fs_read:0
  total_fs_write:0
  total_gpu_energy:0.0
metrics
  t

In [64]:
# list queries
list_queries = evaluate.get_experiment_list_queries()

# list connections
list_nodes = evaluate.get_experiment_list_nodes()
list_dbms = evaluate.get_experiment_list_dbms()
list_connections = evaluate.get_experiment_list_connections()
list_connections_node = evaluate.get_experiment_list_connections_by_node()
list_connections_dbms = evaluate.get_experiment_list_connections_by_dbms()
list_connections_clients = evaluate.get_experiment_list_connections_by_connectionmanagement('numProcesses')
list_connections_gpu = evaluate.get_experiment_list_connections_by_hostsystem('GPU')

# fix some examples:
# first connection, first query, first run
connection = list_connections[0]
numQuery = 1
numRun = 0

pd.set_option("max_rows", None)

In [85]:
query_properties = evaluate.get_experiment_query_properties()
display(Markdown("#### Show Query Template {} - {}".format(numQuery, query_properties[str(numQuery)]['config']['title'])))
print(query_properties[str(numQuery)]['config']['query'])


#### Show Query Template 1 - Pricing Summary Report (TPC-H Q1)

select
l_returnflag,
l_linestatus,
cast(sum(l_quantity) as bigint) as sum_qty,
sum(l_extendedprice) as sum_base_price,
sum(l_extendedprice*(1-l_discount)) as sum_disc_price,
sum(l_extendedprice*(1-l_discount)*(1+l_tax)) as sum_charge,
avg(l_quantity) as avg_qty,
avg(l_extendedprice) as avg_price,
avg(l_discount) as avg_disc,
count(*) as count_order
from
lineitem
where
l_shipdate <= date '1998-12-01' - interval '{DELTA}'  day
group by
l_returnflag,
l_linestatus
order by
l_returnflag,
l_linestatus


In [80]:
connection = list_connections[0]
numQuery = 1
numRun = 0

display(Markdown("#### Show Query {} as run by {} - Run number {}".format(numQuery, connection, numRun)))

query_string = evaluate.get_querystring(numQuery, connection, numRun)
print(query_string)

#### Show Query 1 as run by MonetDB-2-1 - Run number 0

select
l_returnflag,
l_linestatus,
cast(sum(l_quantity) as bigint) as sum_qty,
sum(l_extendedprice) as sum_base_price,
sum(l_extendedprice*(1-l_discount)) as sum_disc_price,
sum(cast(l_extendedprice*(1-l_discount)*(1+l_tax) as double)) as sum_charge,
avg(l_quantity) as avg_qty,
avg(l_extendedprice) as avg_price,
avg(l_discount) as avg_disc,
count(*) as count_order
from
lineitem
where
l_shipdate <= date '1998-12-01' - interval '71'  day
group by
l_returnflag,
l_linestatus
order by
l_returnflag,
l_linestatus


## Show some Measures

### Hardware Metrics

In [8]:
# list all available metrics
pd.DataFrame(monitor.metrics.metrics).T

,query,title
total_cpu_memory,(node_memory_MemTotal_bytes-node_memory_MemFre...,CPU Memory [MiB]
total_cpu_memory_cached,(node_memory_Cached_bytes)/1024/1024,CPU Memory Cached [MiB]
total_cpu_util,100 - (avg by (instance) (irate(node_cpu_secon...,CPU Util [%]
total_gpu_util,sum(dcgm_gpu_utilization),GPU Util [%]
total_gpu_power,sum(dcgm_power_usage),GPU Power Usage [W]
total_gpu_memory,sum(dcgm_fb_used),GPU Memory [MiB]
total_cpu_throttled,sum(rate(container_cpu_cfs_throttled_seconds_t...,CPU Throttle [%]
total_cpu_util_others,sum(irate(container_cpu_usage_seconds_total{{j...,CPU Others [%]
total_cpu_util_s,"sum(container_cpu_usage_seconds_total{{job=""mo...",CPU Util [s]
total_cpu_util_user_s,"sum(container_cpu_user_seconds_total{{job=""mon...",CPU Util User [s]


In [82]:
# get hardware metrics for loading test
df = evaluate.get_loading_metrics('total_cpu_memory')
df = evaluate.get_loading_metrics('total_cpu_util_s')
df = df.T.max().sort_index() - df.T.min().sort_index() # compute difference of counter
print("CPU of Ingestion (via counter)")
pd.DataFrame(df)

CPU of Ingestion (via counter)


,0
DBMS,
MariaDB-1,1526.198404
MariaDB-2,1526.198404
MonetDB-1,1931.119997
MonetDB-2,1931.119997
MySQL-1,1963.867765
MySQL-2,1963.867765
PostgreSQL-1,472.864693
PostgreSQL-2,472.864693


In [10]:
df = evaluate.get_loading_metrics('total_cpu_util')
df = df.T.sum().sort_index() # computer sum of rates
print("CPU of Ingestion (via rate)")
print(df)

CPU of Ingestion (via rate)
DBMS
MariaDB-1       1526.761004
MariaDB-2       1526.761004
MonetDB-1       1887.024304
MonetDB-2       1887.024304
MySQL-1         1962.119638
MySQL-2         1962.119638
PostgreSQL-1     464.746812
PostgreSQL-2     464.746812
dtype: float64


In [11]:
# get hardware metrics per stream
df = evaluate.get_streaming_metrics('total_cpu_memory')
df = evaluate.get_streaming_metrics('total_cpu_util_s')
df = df.T.max().sort_index() - df.T.min().sort_index() # compute difference of counter
print("CPU of Stream (via counter)")
print(df)

CPU of Stream (via counter)
DBMS
MariaDB-1-1       5564.712371
MariaDB-2-1       5497.702421
MonetDB-1-1       1356.184954
MonetDB-2-1       1218.550218
MySQL-1-1         3011.705555
MySQL-2-1         2896.443679
PostgreSQL-1-1    1436.189579
PostgreSQL-2-1    1278.603392
dtype: float64


In [12]:
df = evaluate.get_streaming_metrics('total_cpu_util')
df = df.T.sum().sort_index() # computer sum of rates
print("CPU of Stream (via rate)")
print(df)

CPU of Stream (via rate)
DBMS
MariaDB-1-1       5580.313389
MariaDB-2-1       5519.121645
MonetDB-1-1       1508.432769
MonetDB-2-1       1368.634461
MySQL-1-1         3029.972219
MySQL-2-1         2915.922365
PostgreSQL-1-1    1492.224116
PostgreSQL-2-1    1331.659788
dtype: float64


### Measures, show as table per query

In [13]:
evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='Mean').T

DBMS,MonetDB-2-1,MonetDB-1-1,PostgreSQL-2-1,PostgreSQL-1-1,MariaDB-2-1,MariaDB-1-1,MySQL-2-1,MySQL-1-1
Q1,4517.950883,5563.435273,19866.305729,20039.383417,113481.054475,115989.630678,139650.894833,139965.852845
Q2,351.280993,423.855536,17500.866503,7225.602556,13171.131305,13202.835330,1577.399513,1790.965649
Q3,286.680120,1121.951630,19752.016812,15167.515143,55998.118738,58803.801925,60329.661458,60652.883762
Q4,157.038979,167.959029,3660.092083,63276.056357,10700.938849,10906.610491,10227.802497,10353.898270
Q5,383.102751,369.634689,6885.665454,5783.143584,41958.968865,43173.049879,40186.950882,44849.054096
Q6,129.628404,1785.649381,2886.094683,2917.860596,19288.650417,19219.394048,24026.994607,24070.415718
Q7,3050.601307,2198.463472,13545.375023,11244.580511,33578.747254,35227.129593,24198.993758,27260.769050
Q8,3154.597746,1092.181380,13338.918216,12783.577640,73309.150578,74788.585443,106573.850883,117483.457010
Q9,3963.573644,4918.470218,42587.264411,44068.426727,139187.680818,141097.449787,112492.537863,124488.035032
Q10,803.503769,908.977431,16731.910749,20831.573940,142090.442053,141403.858324,20748.897447,20992.730243


In [ ]:
evaluate.get_aggregated_query_statistics(type='throughput', name='run', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='latency', name='run', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='timer', name='connection', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='timer', name='execution', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='timer', name='datatransfer', query_aggregate='Mean').T

In [ ]:
# measures, show as heatmap (normalized)
evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='factor').T

In [ ]:
# example: filter DBMS
list_filter_dbms = list_connections_dbms['Citus']
evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='factor', dbms_filter=list_filter_dbms).T

In [ ]:
# size of result sets
evaluate.get_total_resultsize()

In [ ]:
# relative size of result sets (minimum=100%), show as heatmap
evaluate.get_total_resultsize_normalized()

In [14]:
# if there was an error, show as heatmap
evaluate.get_total_errors().T

,MariaDB-1-1,MariaDB-2-1,MonetDB-1-1,MonetDB-2-1,MySQL-1-1,MySQL-2-1,PostgreSQL-1-1,PostgreSQL-2-1
Q1,False,False,False,False,False,False,False,False
Q2,False,False,False,False,False,False,False,False
Q3,False,False,False,False,False,False,False,False
Q4,False,False,False,False,False,False,False,False
Q5,False,False,False,False,False,False,False,False
Q6,False,False,False,False,False,False,False,False
Q7,False,False,False,False,False,False,False,False
Q8,False,False,False,False,False,False,False,False
Q9,False,False,False,False,False,False,False,False
Q10,False,False,False,False,False,False,False,False


In [15]:
# if there was a warning, show as heatmap
evaluate.get_total_warnings()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22
MariaDB-1-1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
MariaDB-2-1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MonetDB-1-1,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
MonetDB-2-1,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
MySQL-1-1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
MySQL-2-1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
PostgreSQL-1-1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
PostgreSQL-2-1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [ ]:
# total times benchmarking a query took, show as table
evaluate.get_total_times().T

In [ ]:
# same, normalized to 100% per query, show as stacked area plot
evaluate.get_total_times_normalized().T

In [ ]:
# same, normed to relative to best per query, show as heatmap
evaluate.get_total_times_relative().T

# some dataframes

* showing metrics per dbms by aggregation
* from measures, show as bar plot

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='timer', name='run', query_aggregate='Mean', total_aggregate='Mean')

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='throughput', name='run', query_aggregate='Mean', total_aggregate='Geo')

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='timer', name='run', query_aggregate='factor', total_aggregate='Geo')

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='monitoring', name='total_cpu_memory', query_aggregate='Mean', total_aggregate='Mean')

In [ ]:
# average data obtained from monitoring
# average per second - this differs from mean of mean, because not all queries have the same duration
evaluate.get_survey_monitoring().T

In [ ]:
# host data obtained from config
evaluate.get_survey_hostdata()
# example for merging two dataframes (by index)
df1 = evaluate.get_survey_monitoring()
df2 = evaluate.get_survey_hostdata()
tools.dataframehelper.merge(df1, df2)

In [ ]:
evaluate.get_survey_hostdata()

## Per query details

In [ ]:
# get measures of a query
# 
# df1 = measures for plot, boxplot, histogramm
# df2 = statistics as table
df1, df2 = evaluate.get_measures_and_statistics(numQuery, type='timer', name='execution')
print(df1)
df2

In [ ]:
evaluate.get_measures_and_statistics(numQuery, type='timer', name='execution')
evaluate.get_measures_and_statistics(numQuery, type='timer', name='datatransfer')
evaluate.get_measures_and_statistics(numQuery, type='timer', name='run')
evaluate.get_measures_and_statistics(numQuery, type='timer', name='session')
evaluate.get_measures_and_statistics(numQuery, type='throughput', name='session')

In [ ]:
# colors by dbms
list_connections_dbms = evaluate.get_experiment_list_connections_by_dbms()
connection_colors = evaluate.get_experiment_list_connection_colors(list_connections_dbms)
#connection_colors = evaluate.get_experiment_list_connection_colors(list_connections_node)

In [53]:
numQuery=4
df1,df2=evaluate.get_measures_and_statistics(numQuery, type='timer', name='run', warmup=0)

# Plots
fig = go.Figure()
for i in range(len(df1.index)):
    t = fig.add_trace(go.Scatter(x=df1.T.index, y=df1.iloc[i], name=df1.index[i], line=dict(color=connection_colors[df1.index[i]], width=1)))

fig.update_layout(title_text='Timer Run [ms] - Query {} ({} Measures)'.format(numQuery, len(df1.columns)))
fig.show()

In [17]:
# Heatmap
df = evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='factor')
df = df.sort_index()
df_2=df.applymap(lambda x: ("%.2f" % x))
fig = go.Figure(data=[go.Heatmap(z=df_2.T.values.tolist(),x=df_2.index, y=df_2.columns,colorscale='Reds')])
# optionally fixed size
#t = fig.update_layout(autosize=False,height=2000)
fig.show()

In [45]:
# Bar
df1, df2 = evaluate.get_measures_and_statistics(numQuery, type='timer', name='run')
df = tools.dataframehelper.collect(df2, 'Mean', 'timer_run_mean').sort_index()
fig = go.Figure()
for i in range(len(df.index)):
    t = fig.add_trace(go.Bar(x=[df.index[i]], y=df.iloc[i], name=df.index[i], marker=dict(color=connection_colors[df.index[i]])))

fig.update_layout(title_text='Mean of Timer Run [s] - Query {}'.format(numQuery))
fig.show()

In [46]:
# Boxplots
df1=df1.sort_index()
fig = go.Figure()
for i in range(len(df1.index)):
    t = fig.add_trace(go.Box(y=df1.iloc[i], name=df1.index[i], line=dict(color=connection_colors[df1.index[i]], width=1), boxmean='sd'))

fig.show()


In [ ]:
list_errors = evaluate.get_error(21)
list_errors

In [36]:
df = (evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='Mean').T/1000.0).round(2)
#.sort_index(ascending=False)
#df=df.T
#df=df.round(2)

fig1 = ff.create_annotated_heatmap(
    x=list(df.columns),
    y=list(df.index),
    z=df.values.tolist(),
    showscale=True,
    colorscale='Reds',
    xgap=1,
    ygap=1,
    )

fig1.update_layout(title_text='Timer Run - Mean per Query [s]')
fig1.layout.xaxis.type = 'category'
fig1.layout.yaxis.type = 'category'

fig1.show()

In [41]:
df = (evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='Std Dev').T/1000.0).round(2)
#.sort_index(ascending=False)
#df=df.T
#df=df.round(2)

fig1 = ff.create_annotated_heatmap(
    x=list(df.columns),
    y=list(df.index),
    z=df.values.tolist(),
    showscale=True,
    colorscale='Reds',
    xgap=1,
    ygap=1,
    )

fig1.update_layout(title_text='Timer Run - Std Dev per Query [s]')
fig1.layout.xaxis.type = 'category'
fig1.layout.yaxis.type = 'category'

fig1.show()